In [1]:
import random
from random import sample
import numpy as np
import os
import pickle
from tqdm import tqdm
from collections import OrderedDict
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from scipy.spatial.distance import mahalanobis
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.models import wide_resnet50_2, resnet18
from fast_ixi import FAST_IXI
from utils import read_config

In [2]:
# device setup
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
print('Device: {}'.format(device))

Device: cuda


In [3]:
opt = read_config('config.yaml')
opt

{'dataset': {'ann_path': '/home/mamur/TUM/Seminar/data/splits',
  'path': '/home/mamur/TUM/Seminar/data',
  'name': 'fast_ixi',
  'resize': 128,
  'cropsize': 128,
  'batch_size': 1},
 'model': {'backbone': 'wide_resnet50_2'}}

In [4]:
from preprocess_data import DataHandler
processor = DataHandler(opt)

In [5]:
df = processor.create_records()

['mass_all_neg.csv', 'posttreatment.csv', 'wml_ann.csv', 'edema_neg.csv', 'ixi_normal_train.csv', 'posttreatment_neg.csv', 'intraventricular_neg.csv', 'artefacts_ann.csv', 'lesions.csv', 'normal_train.csv', 'dural_ann.csv', 'mass_all.csv', 'intraventricular.csv', 'normal_val.csv', 'enlarged_ventricles_ann.csv', 'encephalomalacia.csv', 'resection_neg.csv', 'sinus_ann.csv', 'normal_test.csv', 'absent_septum.csv', 'wml_neg.csv', 'mass.csv', 'artefacts_neg.csv', 'ea_mass_neg.csv', 'edema.csv', 'normal_test_ann.csv', 'craniatomy_ann.csv', 'other.csv', 'artefacts.csv', 'intraventricular_ann.csv', 'lesions_ann.csv', 'enlarged_ventricles_neg.csv', 'craniatomy.csv', 'mass_neg.csv', 'resection_ann.csv', 'wml.csv', 'mass_ann.csv', 'encephalomalacia_neg.csv', 'ea_mass.csv', 'dural.csv', 'posttreatment_ann.csv', 'local_neg.csv', 'other_ann.csv', 'lesions_neg.csv', 'edema_ann.csv', 'other_neg.csv', 'encephalomalacia_ann.csv', 'absent_septum_neg.csv', 'resection.csv', 'enlarged_ventricles.csv', 'sinu

In [6]:
df.to_csv('records.csv', index=False)

In [7]:
full_map = df['full_map'].values

In [12]:
# check if all the paths in df['full_map'] are exist
for path in tqdm(full_map):
    if path is None:
        continue
    if not os.path.exists(path):
        print(path)

100%|██████████| 1072/1072 [00:00<00:00, 582270.64it/s]
